In [ ]:
pip install opencv-python-headless numpy torch torchvision scikit-learn

In [ ]:
import os

# Replace with your actual path
data_path = '/content/drive/MyDrive/Project5_Ag_Crop and weed detection/agri_data'
images_dir = os.path.join(data_path, 'images')
labels_dir = os.path.join(data_path, 'labels')

# List files to check paths
print("Images:", os.listdir(images_dir)[:5])  # Print first 5 image files
print("Labels:", os.listdir(labels_dir)[:5])  # Print first 5 label files
# Create dataset.py
dataset_code = """
import os
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class AgriDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.images_dir, self.image_files[idx])
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label_name = os.path.join(self.labels_dir, self.image_files[idx].replace('.jpeg', '.txt'))
        if not os.path.exists(label_name):
            raise FileNotFoundError(f"Label file {label_name} not found.")

        with open(label_name, 'r') as file:
            label = int(file.readline().strip().split()[0])

        if self.transform:
            image = self.transform(image)

        return image, label
"""
with open('dataset.py', 'w') as f:
    f.write(dataset_code)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

def train_val_split(dataset, val_percent=0.2):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_indices, val_indices = train_test_split(indices, test_size=val_percent, random_state=42)

    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)

    return train_dataset, val_dataset

train_dataset, val_dataset = train_val_split(dataset)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
# Create model.py
model_code = """
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*30*30, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32*30*30)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
"""
with open('model.py', 'w') as f:
    f.write(model_code)
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from model import SimpleCNN
from dataset import AgriDataset
from sklearn.model_selection import train_test_split
from torchvision import transforms

# Function to split dataset into training and validation sets
def train_val_split(dataset, val_percent=0.2):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    train_indices, val_indices = train_test_split(indices, test_size=val_percent, random_state=42)

    train_dataset = torch.utils.data.Subset(dataset, train_indices)
    val_dataset = torch.utils.data.Subset(dataset, val_indices)

    return train_dataset, val_dataset

# Hyperparameters
batch_size = 16
num_epochs = 10
learning_rate = 0.001

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define dataset and dataloaders
images_dir = '/content/drive/MyDrive/Project5_Ag_Crop and weed detection/agri_data/images'
labels_dir = '/content/drive/MyDrive/Project5_Ag_Crop and weed detection/agri_data/labels'
dataset = AgriDataset(images_dir=images_dir, labels_dir=labels_dir, transform=transform)
train_dataset, val_dataset = train_val_split(dataset)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, criterion, and optimizer
model = SimpleCNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)
    print(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'model.pth')
# Imports
import torch
import cv2
from torchvision import transforms
from model import SimpleCNN

# Load model function
def load_model(model_path='model.pth'):
    model = SimpleCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set model to evaluation mode
    return model

# Preprocess image function
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 128)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Predict function
def predict(image_path, model):
    image = preprocess_image(image_path)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return 'crop' if preds.item() == 0 else 'weed'

# Load the model
model = load_model('model.pth')

# Predict on a sample image
image_path = '/content/drive/MyDrive/Project5_Ag_Crop and weed detection/agri_data/images/agri_0_6.jpeg'  # Replace with your image path
prediction = predict(image_path, model)
print(f'The predicted class for the image is: {prediction}')
# Imports
import torch
import cv2
from torchvision import transforms
from model import SimpleCNN
import matplotlib.pyplot as plt

# Load model function
def load_model(model_path='model.pth'):
    model = SimpleCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set model to evaluation mode
    return model

# Preprocess image function
def preprocess_image(image_path):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file {image_path} not found.")

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to read image {image_path}. The image file might be corrupted or empty.")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 128)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Predict function
def predict(image_path, model):
    image = preprocess_image(image_path)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return 'crop' if preds.item() == 0 else 'weed', image.cpu().squeeze().permute(1, 2, 0)

# Load the model
model = load_model('model.pth')

# Predict on a sample image
image_path = '/content/weed_1.jpeg'  # Replace with your image path
try:
    prediction, image = predict(image_path, model)
    print(f'The predicted class for the image is: {prediction}')

    # Display the image
    plt.imshow(image)
    plt.title(f'Prediction: {prediction}')
    plt.axis('off')  # Hide the axis
    plt.show()
except Exception as e:
    print(e)
# Imports
import torch
import cv2
from torchvision import transforms
from model import SimpleCNN
import matplotlib.pyplot as plt

# Load model function
def load_model(model_path='model.pth'):
    model = SimpleCNN()
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set model to evaluation mode
    return model

# Preprocess image function
def preprocess_image(image_path):
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file {image_path} not found.")

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to read image {image_path}. The image file might be corrupted or empty.")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((128, 128)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image

# Predict function
def predict(image_path, model):
    image = preprocess_image(image_path)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    return 'crop' if preds.item() == 0 else 'weed', image.cpu().squeeze().permute(1, 2, 0)

# Load the model
model = load_model('model.pth')

# Predict on a sample image
image_path = '/content/crop_1.jpeg'  # Replace with your image path
try:
    prediction, image = predict(image_path, model)
    print(f'The predicted class for the image is: {prediction}')

    # Display the image
    plt.imshow(image)
    plt.title(f'Prediction: {prediction}')
    plt.axis('off')  # Hide the axis
    plt.show()
except Exception as e:
    print(e)